<a href="https://colab.research.google.com/github/Akahaybasutkar/Object_Detector/blob/main/simpleObjectDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the libraries

In [ ]:
!pip install keras

# Importing the libraries and the methods

In [ ]:
import cv2
import os
import numpy as np
from keras.models import load_model
import tensorflow as tf
from keras.applications import VGG16
from keras.applications import VGG16
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

# Main Object Detector

In [ ]:
class ObjectDetector:
    def __init__(self):
        self.image_name = None
        self.model = None

    def load_model(self, user_model=None):
        """
        This is function to load the model, this may be any model that the user
        wants to use(Prebuilt or Custom)

        Parameters:
            model (object): The actual model
        """

        if user_model == None:
            self.model = VGG16(weights='imagenet', include_top=True)
            print("Model created")
        else:
            self.model = user_model

    def load_image(self, image_path):
        """Loads and resizes an image from the specified path.

        This function reads an image from the given file path, resizes it to 672x672 pixels using Lanczos resampling,
        and converts it to a NumPy array.

        Args:
            image_path (str): The path to the image file.

        Returns:
            np.ndarray: The resized image as a NumPy array.

        Raises:
            FileNotFoundError: If the image file does not exist at the specified path.
            cv2.error: If there's an issue loading or resizing the image using OpenCV.
        """

        try:
            image = cv2.imread(image_path)
            if image is None:
                raise FileNotFoundError(f"Image not found at: {image_path}")

            img = cv2.resize(image, (672, 672), interpolation=cv2.INTER_LANCZOS4)
            img_array = np.asarray(img)
            return img_array
        except FileNotFoundError as e:
            print(f"Error: {e}")
            raise e  # Re-raise to be handled elsewhere
        except cv2.error as e:
            print(f"Error loading or resizing image: {e}")
            raise e  # Re-raise to be handled elsewhere


    def preprocess_image(self, image_path, for_use='predections'):
        """Preprocesses an image for use in a model.

        This method loads an image from the given path, resizes it to 224x224 pixels using Lanczos resampling,
        converts it to a NumPy array, and applies model-specific preprocessing.

        Args:
            image_path (str): The path to the image file.
            for_use (str, optional): Specifies how the image will be used.
                                    If 'predictions', it's prepared for model inference. Defaults to 'predictions'.

        Returns:
            np.ndarray: The preprocessed image array, ready for use in a model.

        Raises:
        FileNotFoundError: If the image file does not exist at the specified path.
        cv2.error: If there's an issue loading or resizing the image using OpenCV.
        """

        if for_use == 'predections':
            try:
                img = cv2.imread(image_path)
                if img is None:
                    raise FileNotFoundError(f"Image not found at: {image_path}")
                img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_LANCZOS4)
                img_array = np.asarray(img)
                img_array = np.expand_dims(img_array, axis=0)
                return preprocess_input(img_array)

            except FileNotFoundError as e:
                raise e
            except cv2.error as e:
                raise e


    def break_image(self, inp_image):
        """Breaks a large image into smaller, overlapping 224x224 windows.

        This method systematically crops the input image into overlapping windows of 224x224 pixels.
        The cropped images are saved individually and also returned as a list.

        Args:
            inp_image (np.ndarray): The input image as a NumPy array.

        Returns:
            list[np.ndarray]: A list of cropped image arrays, each of size 224x224.

        Raises:
            TypeError: If the input image is not a NumPy array.
            ValueError: If the input image has dimensions smaller than 224x224.
            OSError: If there's an error creating the output directory.
            cv2.error: If there's an issue writing the cropped images.
        """

        try:
            if not isinstance(inp_image, np.ndarray):
                raise TypeError("Input image must be a NumPy array.")
            if inp_image.shape[0] < 224 or inp_image.shape[1] < 224:
                raise ValueError("Input image dimensions must be at least 224x224.")

            img = inp_image

            window_width = 224
            window_height = 224
            step_size = 112  # Overlap of half the window size
            output_dir = f"/content/{self.image_name}/"

            os.makedirs(output_dir, exist_ok=True)  # Creating the output directory if it doesn't exist
            cropped_images = []

            for y in range(0, img.shape[0] - window_height + 1, step_size):
                for x in range(0, img.shape[1] - window_width + 1, step_size):
                    window = img[y:y + window_height, x:x + window_width]
                    cropped_images.append(window.copy())

                    cell_index = len(cropped_images)
                    filename = f"/content/{self.image_name}/cropped_cell_{cell_index}.jpg"
                    cv2.imwrite(filename, window)

            print(f"Extracted {len(cropped_images)} windows (224*224 images).")
            return cropped_images

        except (TypeError, ValueError) as e:
            raise e
        except OSError as e:
            print(f"Error creating output directory: {e}")
            raise e
        except cv2.error as e:
            print(f"Error saving cropped image: {e}")
            raise e

    def predict_objects(self):
        """Predicts objects in preprocessed image windows.

        This method iterates over the preprocessed image windows, predicts the top object in each window
        using the loaded model, and returns the predictions with confidence scores above a threshold.

        Returns:
            tuple: A tuple containing two dictionaries:
                - output_dic: A dictionary mapping window names (e.g., "window_1") to the top prediction tuples (id, label, confidence).
                - predictions: A list of all top prediction tuples with confidence above the threshold.

        Raises:
            FileNotFoundError: If a preprocessed image file is not found.
            OSError: If there's an issue accessing the image directory.
            Exception: For any other unexpected errors during prediction or decoding.
        """

        predictions = []
        output_dic = {}

        try:
            for i in range(1, 26):
                current_image_path = f"/content/{self.image_name}/cropped_cell_{i}.jpg"

                # Ensure the image file exists
                if not os.path.exists(current_image_path):
                    raise FileNotFoundError(f"Image not found at: {current_image_path}")

                image = self.preprocess_image(current_image_path)
                prediction = self.model.predict(image)
                top_predictions = decode_predictions(prediction, top=1)[0]

                for (id, label, confidence) in top_predictions:
                    if confidence > 0.5:  # Filter for confidence above the threshold
                        output_dic[f"window_{i}"] = top_predictions[0]
                        predictions.append(top_predictions[0])

            return output_dic, predictions

        except FileNotFoundError as e:
            print(f"Error: {e}")
            raise e
        except OSError as e:
            print(f"Error accessing directory: {e}")
            raise e
        except Exception as e:
            print(f"Unexpected error during prediction: {e}")
            raise e


        return output_dic, predections

    def predict_image(self, image_path, img_name):
        """Predicts objects in an image.

        This method loads an image from the given path, breaks it into smaller windows,
        preprocesses each window, and then uses the loaded model to predict the objects
        in each window. It returns the predictions for each window.

        Args:
            image_path (str): The path to the image file.
            img_name (str): The name to be used for saving the image.

        Returns:
            tuple: A tuple containing two dictionaries:
                - output_dic: A dictionary mapping window names (e.g., "window_1") to the top prediction tuples (id, label, confidence).
                - predictions: A list of all top prediction tuples with confidence above the threshold.

        Raises:
            FileNotFoundError: If the specified image file does not exist.
            TypeError: If `image_path` is not a string.
            OSError: If there's an issue accessing the image directory or saving results.
            Exception: For any other unexpected errors during loading, breaking, or predicting.
        """

        try:
            # Check if the image file exists
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image not found at: {image_path}")

            # Check if image_path is a string
            if not isinstance(image_path, str):
                raise TypeError(f"image_path should be a string, not {type(image_path)}")

            self.image_name = img_name
            self.load_model()  # Assuming this method loads your model
            image = self.load_image(image_path)  # Assuming this method loads the image
            self.break_image(image)  # Break image into smaller windows
            preds = self.predict_objects()  # Predict objects in each window
            return preds

        except FileNotFoundError as e:
            print(f"Error: {e}")
            raise e
        except TypeError as e:
            print(f"Error: {e}")
            raise e
        except OSError as e:
            print(f"Error accessing directory or saving results: {e}")
            raise e
        except Exception as e:  # Catch-all for other unexpected errors
            print(f"Unexpected error: {e}")
            raise e


In [ ]:
def process_outputs_summary(preds):
    """Creates a summary of object predictions.

    This function takes a list of prediction tuples (id, label, confidence) and counts the occurrences of each object label.

    Args:
        preds (list): A list of prediction tuples (id, label, confidence) from the model.

    Returns:
        dict: A dictionary where keys are object labels and values are the counts of their occurrences.

    Raises:
        TypeError: If the input `preds` is not a list or if a prediction tuple is not in the expected format.
    """

    # Error handling: Check if input is a list
    if not isinstance(preds, list):
        raise TypeError("Input must be a list of prediction tuples.")

    output_list = {}
    for pred in preds:
        if pred[1] not in output_list:
            output_list[pred[1]] = 1
        elif pred[1] in output_list:
            output_list[pred[1]] += 1
    return output_list


def process_outputs_details(preds):
    """Prints detailed information about object predictions.

    This function takes a dictionary of predictions (where keys are window names and values are prediction tuples)
    and prints a formatted message for each prediction indicating the object found in the corresponding window.

    Args:
        preds (dict): A dictionary mapping window names to prediction tuples (id, label, confidence).
    """
    for key, value in preds.items():
        print(f"Found {value[1]} in {key}")


In [ ]:
image_path = '/content/trafiic2.jpg'

In [ ]:
detector = ObjectDetector()
final_preds, predections = detector.predict_image(image_path, 'car')

Model created
Extracted 25 windows (224*224 images).
1/1 [==============================] - 0s 16ms/step


## Summary of the output

In [ ]:
output = process_outputs_summary(predections)
print(output)

{'cab': 6, 'police_van': 3, 'minivan': 2}


# Detailed output of each window

In [ ]:
process_outputs_details(final_preds)

Found cab in window_1
Found cab in window_2
Found police_van in window_4
Found cab in window_5
Found cab in window_6
Found cab in window_10
Found police_van in window_12
Found police_van in window_13
Found cab in window_14
Found minivan in window_18
Found minivan in window_22
